# Project 2 

Steps:

1. Read multiple csv files into our python program
2. Combine the sources into one file
3. Transform data into desired format - aggregate by "account" column
4. Ingest data into an exisiting Excel 
5. Save the workbook with today's date


## Step by step

In [18]:
# 1
# Read multiple csv files into our python program
# We have done this before :)

In [19]:
# step 1
import os

resource_dir = "../resources/"
def get_target_files(pre_fix):
    """Get the target files."""
    target_files = [file for file in os.listdir(resource_dir) if file.startswith(pre_fix)]
    return target_files

In [20]:
sources = get_target_files("bs")
sources

['bs_equity.csv', 'bs_liabilities.csv', 'bs_assets.csv']

In [21]:
# 2 
# Combine the sources into one file
# We have done this too!

In [22]:
# step 2 
import pandas as pd

def concat_df(file_names):
    """Read in each csv file and concatenate into one dataframe."""
    all_dfs = [pd.read_csv(resource_dir + f) for f in file_names]
    return pd.concat(all_dfs)

In [23]:
combined_df = concat_df(sources)

In [24]:
# 3
# Transform data into desired format
# We want to get aggregated amount by the "account" column

In [25]:
combined_df.head()

,field,account,amount
0,Preferred equity,Preferred equity,0
1,Common stock,Share capital and APIC,400
2,Additional paid in capital,Share capital and APIC,100
3,Retained earnings,Retained earnings,240
0,Account payable,Accounts payable and accrued expenses,300


In [26]:
# Group by the account and apply the "sum" method to the amount
sum_by_account = combined_df.groupby("account")['amount'].sum()
sum_by_account

account
Accounts payable and accrued expenses                     400
Accounts receivable  [I]                                  600
Cash and cash equivalents  [D]                            800
Deferred income taxes                                      20
Deferred income taxes liabilities                         200
Inventories  [C]                                          950
Less accumulated depreciation                             400
Loans payable and current portion long-term debt  [H]      30
Long-term debt  [G]                                      1000
Other assets                                              290
Other long term liabilities                               200
Preferred equity                                            0
Property, plant and equipment at cost                    1000
Retained earnings                                         240
Share capital and APIC                                    500
Short-term investments                                    200


In [27]:
sum_by_account.to_dict()  # Series has to_dict method to convert to dictionary

{'Accounts payable and accrued expenses': 400,
 'Accounts receivable  [I]': 600,
 'Cash and cash equivalents  [D]': 800,
 'Deferred income taxes': 20,
 'Deferred income taxes liabilities': 200,
 'Inventories  [C]': 950,
 'Less accumulated depreciation': 400,
 'Loans payable and current portion long-term debt  [H]': 30,
 'Long-term debt  [G]': 1000,
 'Other assets': 290,
 'Other long term liabilities': 200,
 'Preferred equity': 0,
 'Property, plant and equipment at cost': 1000,
 'Retained earnings': 240,
 'Share capital and APIC': 500,
 'Short-term investments': 200,
 'other current liabilities': 1690}

In [28]:
# step 3
def transform_data(df):
    """Some transformation."""
    sum_by_account = df.groupby("account")['amount'].sum().to_dict()
    return sum_by_account

In [29]:
account_val_map = transform_data(combined_df)

In [30]:
# 4
# Ingest data into an exisiting Excel 

In [31]:
# load excel file - we get back a "Workbook" object
import openpyxl as xl
wb = xl.load_workbook(resource_dir + 'balancesheet.xlsx')

In [32]:
# get the "Worksheet" object
st = wb['BS'] 

In [33]:
# We can loop through column "A" and if we find the account name that matches
# then we assign the value to column "B" in the same row
for cell in st["A"]:
    if cell.value in account_val_map:
        row_num = cell.row  # get row number
        # set the value for column B
        st["B" + str(row_num)].value = account_val_map[cell.value]
        
# do the same thing with column E
for cell in st["E"]:
    if cell.value in account_val_map:
        row_num = cell.row  # get row number
        # set the value for column F
        st["F" + str(row_num)].value = account_val_map[cell.value]

In [36]:
# Step 4
def upadte_balancesheet(balancesheet, account_val_map):
    wb = xl.load_workbook(resource_dir + balancesheet)
    st = wb['BS'] 
    for cell in st["A"]:
        if cell.value in account_val_map:
            row_num = cell.row  # get row number
            # set the value for column B
            st["B" + str(row_num)].value = account_val_map[cell.value]
        
    # do the same thing with column E
    for cell in st["E"]:
        if cell.value in account_val_map:
            row_num = cell.row  # get row number
            # set the value for column F
            st["F" + str(row_num)].value = account_val_map[cell.value]
            
    return wb    

In [37]:
my_bs = upadte_balancesheet('balancesheet.xlsx', account_val_map)
my_bs

In [38]:
# 5
# Save the file with today's date

In [39]:
from datetime import datetime  # use python datetime module

today = datetime.now()  # will give you a datetime object
today

datetime.datetime(2021, 1, 28, 0, 3, 32, 486159)

In [40]:
# use strftime method to parse the string value and format it
today_str = today.strftime("%Y-%m-%d")
today_str

'2021-01-28'

In [41]:
# step 5
def get_today():
    today = datetime.now()
    today_str = today.strftime("%Y-%m-%d")
    return today_str  

In [44]:
# put everything together
def main():
    sources = get_target_files("bs")
    combined_df = concat_df(sources)
    account_map = transform_data(combined_df)
    bs_wb = upadte_balancesheet("balancesheet.xlsx", account_map)
    today_str = get_today()
    bs_wb.save(f"bs_{today_str}.xlsx")
    print("Done!!")

In [45]:
main()

Done!!
